In [1]:
import numpy as np
import matplotlib.pyplot as plt
import spekpy as sp
from scipy.integrate import quad, cumulative_trapezoid
import os
import pandas as pd
# from datetime import date, time, datetime

import time
import datetime
from nptdms import TdmsFile
from uncertainties import unumpy as unp
from uncertainties import ufloat
import numpy as np
import time

In [9]:
thurs = {'started': array(["11:35AM"]), "refilled":array(["2:48PM",]), 'mixed': array(['4:20PM', '5:02PM', '5:24PM']), 'date' : '2025-03-13', 'meterWARM' : True}
fri = {'started': array(["10:08AM"]), "refilled":array(["2:24PM",]), 'mixed': array(['2:29PM', '3:47PM', '5:05PM']), 'date' : '2025-03-14', 'meterWARM' : False}
mon = {'started': array(["11:00AM"]), "refilled":array(["3:20PM",]), 'mixed': array(['1:04PM']), 'date' : '2025-03-17', 'meterWARM' : True}
tue = {'started': array(["10:03AM"]), "refilled":array(["2:31PM",]), 'mixed': array(['2:00PM']), 'date' : '2025-03-18', 'meterWARM' : True}
wed = {'started': array(["11:44AM"]), "refilled":array(["4:35PM",]), 'mixed': array(['1:56PM', '3:46PM']), 'date' : '2025-03-19', 'meterWARM' : False}
alldaydict = array([thurs, fri, mon , tue, wed])

def daynotes(alldaydict):
    alldaydict_old = alldaydict
    alldaydict_new = deepcopy(alldaydict)
    for daydict in alldaydict_new:
        timekeys = ['started', 'mixed', 'refilled']
        daydate = daydict['date']
        for tkey in timekeys:
            daydict[tkey] = pd.to_datetime(daydate + ' ' + daydict[tkey])

        for tkey in ['mixed', 'refilled']:
            daydict[tkey] = daydict[tkey] - daydict['started']

    return alldaydict_old, alldaydict_new
from pandas import TimedeltaIndex, DatetimeIndex
# alldaydict_old, alldaydict_new = daynotes(alldaydict)

alldaydict_new = array([{'started': DatetimeIndex(['2025-03-13 11:35:00'], dtype='datetime64[ns]', freq=None), 'refilled': TimedeltaIndex(['0 days 03:13:00'], dtype='timedelta64[ns]', freq=None), 'mixed': TimedeltaIndex(['0 days 04:45:00', '0 days 05:27:00', '0 days 05:49:00'], dtype='timedelta64[ns]', freq=None), 'date': '2025-03-13', 'meterWARM': True},
       {'started': DatetimeIndex(['2025-03-14 10:08:00'], dtype='datetime64[ns]', freq=None), 'refilled': TimedeltaIndex(['0 days 04:16:00'], dtype='timedelta64[ns]', freq=None), 'mixed': TimedeltaIndex(['0 days 04:21:00', '0 days 05:39:00', '0 days 06:57:00'], dtype='timedelta64[ns]', freq=None), 'date': '2025-03-14', 'meterWARM': False},
       {'started': DatetimeIndex(['2025-03-17 11:00:00'], dtype='datetime64[ns]', freq=None), 'refilled': TimedeltaIndex(['0 days 04:20:00'], dtype='timedelta64[ns]', freq=None), 'mixed': TimedeltaIndex(['0 days 02:04:00'], dtype='timedelta64[ns]', freq=None), 'date': '2025-03-17', 'meterWARM': True},
       {'started': DatetimeIndex(['2025-03-18 10:03:00'], dtype='datetime64[ns]', freq=None), 'refilled': TimedeltaIndex(['0 days 04:28:00'], dtype='timedelta64[ns]', freq=None), 'mixed': TimedeltaIndex(['0 days 03:57:00'], dtype='timedelta64[ns]', freq=None), 'date': '2025-03-18', 'meterWARM': True},
       {'started': DatetimeIndex(['2025-03-19 11:44:00'], dtype='datetime64[ns]', freq=None), 'refilled': TimedeltaIndex(['0 days 04:51:00'], dtype='timedelta64[ns]', freq=None), 'mixed': TimedeltaIndex(['0 days 02:12:00', '0 days 04:02:00'], dtype='timedelta64[ns]', freq=None), 'date': '2025-03-19', 'meterWARM': False}],
      dtype=object)

alldaydict_old = [{'started': array(['11:35AM'], dtype='<U7'), 'refilled': array(['2:48PM'], dtype='<U6'), 'mixed': array(['4:20PM', '5:02PM', '5:24PM'], dtype='<U6'), 'date': '2025-03-13', 'meterWARM': True}, {'started': array(['10:08AM'], dtype='<U7'), 'refilled': array(['2:24PM'], dtype='<U6'), 'mixed': array(['2:29PM', '3:47PM', '5:05PM'], dtype='<U6'), 'date': '2025-03-14', 'meterWARM': False},  {'started': array(['11:00AM'], dtype='<U7'), 'refilled': array(['3:20PM'], dtype='<U6'), 'mixed': array(['1:04PM'], dtype='<U6'), 'date': '2025-03-17', 'meterWARM': True}, {'started': array(['10:03AM'], dtype='<U7'), 'refilled': array(['2:31PM'], dtype='<U6'), 'mixed': array(['2:00PM'], dtype='<U6'), 'date': '2025-03-18', 'meterWARM': True}, {'started': array(['11:44AM'], dtype='<U7'), 'refilled': array(['4:35PM'], dtype='<U6'), 'mixed': array(['1:56PM', '3:46PM'], dtype='<U6'), 'date': '2025-03-19', 'meterWARM': False}]

In [10]:
def tdelta_to_hrmin(relarr):
    return [str(relarr[i])[7:12] for i in range(len(relarr))]

file_name = 'Further Testing - Run Information.txt'
f = open(file_name, "w")
f.write("This file contains information about the data in the 'Further Tests' Folder. \nThe start date and time of the experiment on each day is given, along with the relative times at which the liquid nitrogen was either refilled or mixed. \n\n")
dadd = ['Thurs', 'Fri', 'Mon', 'Tue', 'Wed']
for i, (day_abs, day_rel) in enumerate(zip(alldaydict_old, alldaydict_new)):
    f.write(f"Day {i+1}: {dadd[i]} {day_rel['started'].date.astype(str)}\n")
    f.write(f"Time Started: {day_rel['started'].time.astype(str)[0]}\n")

    mixtimes = tdelta_to_hrmin(day_rel['mixed'])
    filltimes = tdelta_to_hrmin(day_rel['refilled'])
    
    f.write(f"Times Filled: {filltimes}\n")
    
    f.write(f"Times Mixed: {mixtimes}\n")
    
    
    f.write("\n")
f.close()

f = open(file_name, "r")
print(f.read())
f.close()

This file contains information about the data in the 'Further Tests' Folder. 
The start date and time of the experiment on each day is given, along with the relative times at which the liquid nitrogen was either refilled or mixed. 

Day 1: Thurs ['2025-03-13']
Time Started: 11:35:00
Times Filled: ['03:13']
Times Mixed: ['04:45', '05:27', '05:49']

Day 2: Fri ['2025-03-14']
Time Started: 10:08:00
Times Filled: ['04:16']
Times Mixed: ['04:21', '05:39', '06:57']

Day 3: Mon ['2025-03-17']
Time Started: 11:00:00
Times Filled: ['04:20']
Times Mixed: ['02:04']

Day 4: Tue ['2025-03-18']
Time Started: 10:03:00
Times Filled: ['04:28']
Times Mixed: ['03:57']

Day 5: Wed ['2025-03-19']
Time Started: 11:44:00
Times Filled: ['04:51']
Times Mixed: ['02:12', '04:02']


